In [1]:
import logging
from pathlib import Path
import requests
import io
import zipfile
from requests.exceptions import HTTPError
import glob
import pandas as pd
import yaml
from pandas import json_normalize

def get_repo_urls(filename, branch_name):
    git_url = 'https://github.com/'
    file_name = f'{branch_name}.zip'
    suffix_string = 'archive/'+ file_name
    with open(filename, 'r', encoding='UTF-8') as f:
        repos = [git_url + line.rstrip() for line in f]
        repo_archive_urls = [line + suffix_string for line in repos]
    
    return repo_archive_urls

def download_git_archive(git_url, output_dir):
    print(f"Downloading from {git_url}, may take few mins..")
    try:
        r = requests.get(git_url)
        repo_zip = io.BytesIO(r.content)
        archive = zipfile.ZipFile(repo_zip, mode="r")
        for file in archive.namelist():
            archive.extract(file, path=output_dir)
        print("Downloaded and Extracted Files successfully")
    except HTTPError as http_err:
        warnings.warn(f"HTTP error occurred trying to download from Github: {http_err}")
        
def get_sentinel_queries_from_github(git_url, outputdir):
    print(f"Downloading from Azure Sentinel Github, may take 2-3 mins..")
    try:
        r = requests.get(git_url)
        repo_zip = io.BytesIO(r.content)
        archive = zipfile.ZipFile(repo_zip, mode="r")
        # Only extract Detections and Hunting Queries Folder
        for file in archive.namelist():
            if file.startswith(
                (
                    "Azure-Sentinel-master/Detections/",
                    "Azure-Sentinel-master/Hunting Queries/",
                    "Azure-Sentinel-master/Solutions/"
                )
            ):
                archive.extract(file, path=outputdir)
        print("Downloaded and Extracted Files successfully")
    except HTTPError as http_err:
        warnings.warn(f"HTTP error occurred trying to download from Github: {http_err}")
        
def parse_yaml(parent_dir, child_dir):

    sentinel_repourl = "https://github.com/Azure/Azure-Sentinel/blob/master"

    # Collect list of files recusrively uinder a folder
    yaml_queries = glob.glob(f"{parent_dir}/{child_dir}/**/*.yaml", recursive=True)
    df = pd.DataFrame()

    # Recursively load yaml Files and append to dataframe
    for query in yaml_queries:
        with open(query, "r", encoding="utf-8", errors="ignore") as f:
            parsed_yaml_df = json_normalize(yaml.load(f, Loader=yaml.FullLoader))
            parsed_yaml_df["DetectionURL"] = query.replace(parent_dir, sentinel_repourl)
            frames = [df, parsed_yaml_df]
            df = pd.concat(frames, ignore_index=True, sort=True)

    if child_dir == "Detections":
        df["DetectionType"] = "Analytics"
    elif child_dir == "Hunting Queries":
        df["DetectionType"] = "Hunting"
    elif child_dir == "Solutions":
        df["DetectionType"] = "Solutions"

    df["DetectionService"] = "Azure Sentinel Community Github"

    return df

In [2]:
repo_archive_urls = get_repo_urls('repo.conf', 'main')
#Set output dir
output_dir = Path.cwd()

#download git repos
for url in repo_archive_urls:
    download_git_archive(url, output_dir)

Downloaded and Extracted Files successfully
Downloaded and Extracted Files successfully


In [ ]:
#Download and Parses Microsoft Sentinel Repos
azsentinel_git_url = "https://github.com/Azure/Azure-Sentinel/archive/master.zip"
get_sentinel_queries_from_github(git_url=azsentinel_git_url, outputdir=output_dir)

In [ ]:
tmp_path = str(Path.cwd())

base_dir = tmp_path + "/Azure-Sentinel-master"
columns = ['id', 'description', 'DetectionURL','query','tags','tactics','techniques']
detections_df = parse_yaml(parent_dir=base_dir, child_dir="Detections")
detections_df = detections_df[columns]
# hunting_df = parse_yaml(parent_dir=base_dir, child_dir="Hunting Queries")
# hunting_df = hunting_df[columns]
solutions_df = parse_yaml(parent_dir=base_dir, child_dir="Solutions")
solutions_df = solutions_df[columns]

frames = [detections_df, solutions_df]
sentinel_df = pd.concat(frames, ignore_index=True, sort=True)

In [ ]:
sentinel_df.head()

In [226]:
def parse_markdown():
    df = pd.DataFrame()
    
    # Collect list of files recursively under a folder
    parent_dir = tmp_path + "/KQL_Intune-main"
    md_queries = glob.glob(f"{parent_dir}/**/*.md", recursive=True)
    parent_dir = tmp_path + "/Sentinel-Queries-main"
    md_queries = md_queries + glob.glob(f"{parent_dir}/**/*.md", recursive=True)
    
    df = pd.DataFrame(columns=['title', 'kql_query'])
    
    # Recursively load md Files and append to dataframe
    for query in md_queries:
        print("loading file:", query)
        lines = Path(query).read_text(encoding="utf-8").split('\n')
#         print(lines)
#         kql_lines = re.findall("```kql([^```]*)", lines)
#         ret.extend(kql_lines)
        ct = 0
        kql = False
        kql_collect = []
        title_collect = []
        cur_kql = []
        title = "n/a"
        while ct < len(lines):
            if kql:
                cur_kql.append(l[ct])
            if (lines[ct].startswith("#") and lines[ct+2] == "```kql"):
        #         print(l[ct])
                kql = True
                title = lines[ct]
            elif  (lines[ct] == "```kql"):
                kql = True
            elif lines[ct] == "```":
                kql = False
                cur_kql = "\n".join(cur_kql)
                kql_collect.append(cur_kql)
                title_collect.append(title)
                title = "n/a"
                cur_kql = []
            ct+=1
        test_df = pd.DataFrame(list(zip(title_collect, kql_collect)), columns=['title', 'kql_query'])
#         df.append(test_df)
        df = pd.concat([df, test_df])
    
    return df
    

In [227]:
md_queries = parse_markdown()

loading file: C:\Users\jannieli\OneDrive - Microsoft\Documents\hackathon2022/KQL_Intune-main\README.md
loading file: C:\Users\jannieli\OneDrive - Microsoft\Documents\hackathon2022/KQL_Intune-main\Azure Workbook\readme.md
loading file: C:\Users\jannieli\OneDrive - Microsoft\Documents\hackathon2022/KQL_Intune-main\Query Pack\readme.md
loading file: C:\Users\jannieli\OneDrive - Microsoft\Documents\hackathon2022/Sentinel-Queries-main\README.md
loading file: C:\Users\jannieli\OneDrive - Microsoft\Documents\hackathon2022/Sentinel-Queries-main\Azure AD Abuse Detection\README.md
loading file: C:\Users\jannieli\OneDrive - Microsoft\Documents\hackathon2022/Sentinel-Queries-main\Functions\README.md
loading file: C:\Users\jannieli\OneDrive - Microsoft\Documents\hackathon2022/Sentinel-Queries-main\Query Pack\README.md
loading file: C:\Users\jannieli\OneDrive - Microsoft\Documents\hackathon2022/Sentinel-Queries-main\Sentinel vs Advanced Hunting\README.md
loading file: C:\Users\jannieli\OneDrive - Mi

In [228]:
len(md_queries.index)

100

In [229]:
display(md_queries)

,title,kql_query
0,n/a,SigninLogs\n| where TimeGenerated > ago(14d)\n...
1,n/a,SigninLogs\n```
2,n/a,SigninLogs\n| where TimeGenerated > ago(14d)\n```
3,n/a,SigninLogs\n| where TimeGenerated > ago(14d)\n...
4,n/a,SigninLogs\n| where TimeGenerated > ago(14d)\n...
...,...,...
3,n/a,Microsoft Sentinel will then run through your ...
4,n/a,"| where AppDisplayName == ""Microsoft Teams""\n`..."
5,n/a,| where TimeGenerated > ago(14d)\n| where User...
6,n/a,"That is how you build queries, now the basics...."


In [230]:
md_queries[md_queries['title'] != 'n/a']

,title,kql_query
0,### Detection Query (User as actor),\nWe want to use KQL to create accurate and ef...
1,### Detection Query (User as actor),So first we have chosen our SigninLogs table.\...
2,### Detection Query (User as actor),Then we look for only logs where the ResultTyp...
3,### Detection Query (Service principal as actor),SigninLogs\n| where TimeGenerated > ago(14d)\n...
4,### Detection Query (User as actor),```\n\nIs much more efficient than searching f...
5,### Detection Query (Service principal as actor),SigninLogs\n| where TimeGenerated > ago(14d)\n...
6,### Detection Query (User as actor),SigninLogs\n| where TimeGenerated between (ago...
7,### Detection Query (Service principal as actor),SigninLogs\n| where TimeGenerated between (ago...
8,### Detection Query (User as actor),"\nInstead of equals, we can also use contains...."
9,### Detection Query (Service principal as actor),```kql\nSigninLogs\n| where TimeGenerated > ag...


In [ ]:
tmp_path = str(Path.cwd())
csv_files = glob.glob(os.path.join(path, "*.csv"))